In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer
from wordvecs import WordVectors, EmbeddingLayer
import json

theano.config.floatX = 'float32'
theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki1.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
8917/9915.

0.8993444276348966

In [7]:

wordvectors = WordVectors(
    fname="/data/matthew/GoogleNews-vectors-negative300.bin",
    negvectors=False
)

In [8]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
    page_redirects = json.load(f)

In [9]:
from wikireader import WikiRegexes, WikipediaReader

In [10]:
def PreProcessedQueries(wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wordvec.tokenize(sur)
            for link in v['vals'].keys():
                wordvec.tokenize(link)
                tt = WikiRegexes.convertToTitle(link)
                #self.wordvecs.tokenize(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

    class GetWikipediaWords(WikipediaReader, WikiRegexes):

        def readPage(ss, title, content):
            tt = ss.convertToTitle(title)
            if tt in queried_pages:
                cnt = ss._wikiToText(content)
                page_content[tt] = wordvec.tokenize(cnt)

    GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        
        
    return PreProcessedQueriesCls

In [11]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [29]:
class EntityVectorLinkExp(basePreProcessedQueries):
    
    batch_size = 20000
    num_training_items = 200000
    
    def __init__(self): #, wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
        #self.wordvecs = wordvec
        #self.queries = queries
        self.sentence_length = self.wordvecs.sentence_length
        self.num_words_to_use_conv = 3
        #self.redirects = redirects
        #self.page_content = {}
        #self.wikipedia_dump_fname = wikipedia_dump_fname
        
        #self._process_queries()
        
        self._setup()
        
#     def _process_queries(self):
#         queried_pages = set()
#         for docs, q in self.queries.iteritems():
#             self.wordvecs.tokenize(docs)
#             for sur, v in q.iteritems():
#                 self.wordvecs.tokenize(sur)
#                 for link in v['vals'].keys():
#                     self.wordvecs.tokenize(link)
#                     tt = WikiRegexes.convertToTitle(link)
#                     #self.wordvecs.tokenize(tt)
#                     queried_pages.add(tt)

#         added_pages = set()
#         for title in queried_pages:
#             if title in self.redirects:
#                 #self.wordvecs.tokenize(self.redirects[title])
#                 added_pages.add(self.redirects[title])
#         queried_pages |= added_pages
        
#         self.queried_pages = queried_pages
                
#         class GetWikipediaWords(WikipediaReader, WikiRegexes):
            
#             def readPage(ss, title, content):
#                 tt = ss.convertToTitle(title)
#                 if tt in queried_pages:
#                     cnt = ss._wikiToText(content)
#                     self.page_content[tt] = self.wordvecs.tokenize(cnt)
        
#         GetWikipediaWords(self.wikipedia_dump_fname).read()
               
        
    def _setup(self):
        self.x_document_input = T.imatrix('x_sent')
        self.x_surface_text_input = T.imatrix('x_surface')
        self.x_target_input = T.imatrix('x_target')
        self.x_document_id = T.ivector('x_sent_id')
        self.x_link_id = T.ivector('x_link_id')
        self.y_score = T.vector('y')
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        
        self.document_l = lasagne.layers.InputLayer(
            (None,self.sentence_length), 
            input_var=self.x_document_input
        )
    
        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.document_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=350,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='document_conv1',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.document_max_l = lasagne.layers.Pool2DLayer(
            self.document_conv1_l,
            name='document_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='max',
        )

        self.document_dens1 = lasagne.layers.DenseLayer(
            self.document_max_l,
            num_units=300,
            name='doucment_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_drop1 = lasagne.layers.DropoutLayer(
            self.document_dens1,
            p=.25,
        )
        
        document_output_length = 250
        
        self.document_dens2 = lasagne.layers.DenseLayer(
            self.document_drop1,
            num_units=document_output_length,
            name='document_dens2',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_output = lasagne.layers.get_output(self.document_dens2)
                
        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length), 
            input_var=self.x_surface_text_input
        )
        
        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=350,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_conv1_l,
            name='surface_dens1',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.surface_drop1 = lasagne.layers.DropoutLayer(
            self.surface_dens1,
            p=.25,
        )
        
        self.surface_dens2 = lasagne.layers.DenseLayer(
            self.surface_drop1,
            name='surface_dens2',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_aligned_l = InputLayer(
            (None, document_output_length),
            input_var=self.document_output[self.x_document_id,:]
        )
        
        self.source_l = lasagne.layers.ConcatLayer(
            [self.document_aligned_l, self.surface_dens2]
        )
        
        self.source_dens1 = lasagne.layers.DenseLayer(
            self.source_l,
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )
        
        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=300,
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_out = lasagne.layers.get_output(self.source_dens2)
        
        self.target_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length), 
            input_var=self.x_target_input
        )
        
        self.target_embedding_l = EmbeddingLayer(
            self.target_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_embedding_l,
            name='target_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_conv1_l,
            name='target_dens1',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )
        
        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_dens2',
            num_units=300,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_out = lasagne.layers.get_output(self.target_dens2)
        
        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()
        
        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)
    
        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) * 
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
    
        self.all_params = (
            lasagne.layers.get_all_params(self.target_dens2) + 
            lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.document_dens2)
        )
        
        # weight the positive samples more since there are fewer of them,
        # freaking hack
        self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))
        
        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)
        
        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)
        
        self.train_func = theano.function(
            [self.x_document_input,
             self.x_surface_text_input, self.x_document_id,
             self.x_target_input, self.x_link_id, self.y_score],
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates
        )
        
        self.test_func = theano.function(
            [self.x_document_input,
             self.x_surface_text_input, self.x_document_id,
             self.x_target_input, self.x_link_id, self.y_score],
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
        )
        
    def reset_accums(self):
        self.current_documents = []
        self.current_surface_text = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_id = []
        self.current_target_goal = []
        self.learning_targets = []
        
    def compute_batch(self, isTraining=True):
        if isTraining:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        
        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] is not isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_text)
                self.current_surface_text.append(self.wordvecs.tokenize(surtxt))
                for target in targets['vals'].keys():
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    self.current_target_input.append(cnt)  # page_content already tokenized
                    self.current_target_id.append(surid)
                    self.current_target_goal.append(isGold)
                    self.learning_targets.append((targets, target))
            if len(self.current_target_id) > self.batch_size:
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links
        
        if len(self.current_target_id) > 0:
            self.run_batch(func)
            
        return self.total_loss / self.total_links
        
    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            self.current_documents,
            self.current_surface_text, self.current_link_id,
            self.current_target_input, self.current_target_id, self.current_target_goal
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            l[0]['vals'][ l[1] ] = res_vec[i]
        self.reset_accums()
        
    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')
        

        
queries_exp = EntityVectorLinkExp() 

In [30]:
queries_exp.check_params()

In [31]:
exp_results = []

for i in xrange(5):
    exp_results.append((i, queries_exp.compute_batch()))

In [ ]:
queries_exp.compute_batch(False)

In [32]:
exp_results

[(0, 1.162608259072301),
 (1, 1.1083744940828513),
 (2, 1.1107771458564479),
 (3, 1.0799693977008746),
 (4, 1.0744760715970667)]

In [33]:
queries.values()[0]

{u'1921': {u'gold': u'1921',
  u'training': True,
  u'vals': {u'-NIL-': 0.18261611,
   u'144th New York State Legislature': 0.41459617,
   u'1921': 0.42078227,
   u'1921 APFA season': 0.41421959,
   u'1921 Green Bay Packers season': 0.4248966,
   u'1921 Indianapolis 500': 0.37826195,
   u'1921 NFL season': 0,
   u'1921 VFL season': 0.3788352,
   u'1921 World Series': 0.46165341,
   u'1921 college football season': 0.4159075,
   u'1921 in Canada': 0.44230342,
   u'1921 in aviation': 0.42985418,
   u'1921 in film': 0.44654363,
   u'1921 in literature': 0.41353869,
   u'1921 in poetry': 0.46225181,
   u'1921 in the United States': 0.45401299,
   u'Canadian federal election, 1921': 0.42354047,
   u'Irish elections, 1921': 0.44229493,
   u'Norwegian parliamentary election, 1921': 0.43300906,
   u'XXNILXX': 0}},
 u'1984': {u'gold': u'1984',
  u'training': True,
  u'vals': {u'-NIL-': 0.190595,
   u'1984': 0.50053537,
   u'1984 Grand Prix motorcycle racing season': 0.38998544,
   u'1984 NBA dr

In [ ]:
gg_res = queries_exp.test_func(
    queries_exp.current_documents, 
    queries_exp.current_surface_text, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [ ]:
np.isnan(gg_res[0]).any()

In [ ]:
gg_res[2].mean()

In [ ]:
gg_res[0].mean()

In [ ]:
[(v, np.isnan(v.get_value(borrow=True)).all()) for v in queries_exp.all_params]

In [ ]:
queries_exp.total_links

In [ ]:
theano.printing.pydotprint(T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params))

In [ ]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_score],
            T.grad(queries_exp.loss_vec.mean(), lasagne.layers.get_all_params(queries_exp.target_dens2)),
#           T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params),
    #             [queries_exp.target_out, queries_exp.source_aligned_l, 
#              T.dot(queries_exp.target_out, queries_exp.source_aligned_l.T).diagonal(),
#              queries_exp.target_out.norm(2, axis=1) * queries_exp.source_aligned_l.norm(2, axis=1),
#              T.batched_dot(queries_exp.target_out, queries_exp.source_aligned_l),
#              lasagne.layers.get_output(queries_exp.target_dens2),
#              queries_exp.target_out.norm(2, axis=1),
#              #T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params)
#             ],
        #[queries_exp.res_l, queries_exp.loss_vec.sum(), queries_exp.loss_vec],
    on_unused_input='ignore',
    mode='DebugMode'
        )

In [ ]:
try:
    gg_grad_res = gg_func(
        queries_exp.current_documents,
        queries_exp.current_surface_text, queries_exp.current_link_id,
        queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
    )
except Exception as e:
    eeee = e

In [ ]:
[np.isnan(v).any() for v in gg_grad_res]

In [ ]:
eeee

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
gg_res = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_text, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
np.isnan(gg_res[5]).any()

In [ ]:
gg_res[2]

In [ ]:
gg_res[0].shape, gg_res[1].shape, gg_res[2].shape, gg_res[3].shape, gg_res[4].shape

In [ ]:
np.inner(gg_res[0], gg_res[1]).shape

In [ ]:
gg_res[0] * gg_res[1]

In [ ]:
aa = np.dot(gg_res[0], gg_res[1].T)

In [ ]:
aa.diagonal().shape

In [ ]:
gg_res[2]

In [ ]:
len(queries_exp.queried_pages)

In [ ]:
len(queries_exp.page_content)

In [ ]:
len(queries_exp.current_surface_text)

In [ ]:
len(queries_exp.current_link_id)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
exp_results = []

for i in xrange(5):
    exp_results.append((i, queries_exp.compute_batch()))

In [ ]:
exp_results

In [ ]:
queries.values()

In [ ]:
queries_exp.total_links

In [ ]:
queries_exp.total_loss / queries_exp.total_links

In [ ]:
queries_exp.total_loss

In [ ]:
len(queries_exp.current_target_id)

In [ ]:
queries_exp.compute_batch()

In [ ]:
%time queries_exp.run_batch(queries_exp.test_func)